In [186]:
from avipy import qty, atmosphere as atm
import math

# Individual assignment on Takeoff Weight Limitations

Make a TL (take-off weight limitation) table for a B777-300ER  commercial aircraft for Hong Kong's Chek Lap Kok International Airport runway 25C with the environmental condition of 17 degrees Celsius and no wind. The B777-300ER has two certified take-off flap settings of 5 and 15, 15 degrees will be used for the calculations. The aircraft is equipped with GE90-115B engines that deliver a thrust of 104,500 lbs per engine at sea level. The engine is flat rated up to ISA+15ºC. Above ISA+15° the thrust decreases by 0.4% per degree. Explain and make a conclusion of your results!

## Runway
- ASDA and TODA are 2900 m
- Elevation 0 ft
- No runway slope
- Obstacle 1600 m after end of the runway at 103 ft
- Coefficient of friction, $ \mu = 0.027 $

## Aircraft
- MTOW, $ m = 351534 kg $
- wingspan, $ b = 64.8 m $
- wing area, $ S = 436.8 m^2 $
- oswald factor, $ e = 0.77 $
- drag coefficient at flap 15, $ C_{D_0} = 0.068 $
- Stall speed at MTOW with flaps 15, $ v_{stall} = 154.5 kts $
- $ v_2 = 1.18 \cdot v_{stall} $
- $ v_{lof} = 1.15 v_{stall} $
- Coefficient of friction with full brakes, $ \mu = 0.56 $

## Assumptions
- During climb to 1500 ft, air pressure remains constant to ISA at sea level.
- Engine thrust is constant during takeoff.
- N-1 situation arises at lift-off.
- For ASDR, no need to consider pilot's reaction time => $ v_1 == v_{lof} $.
- At $ v_{lof} $, the ground run instantly changes into the take-off arc.

In [187]:
toda = qty.Distance(2900)
asda = qty.Distance(2900)
height_b = qty.Distance.Ft(35)
mu = 0.027
mu_brake = 0.56
temp = qty.Temperature.Celsius(17)
pressure = qty.Pressure.Hpa(1013.25)
obstacle_distance = qty.Distance(1600)
obstacle_height = qty.Distance.Ft(103)

mtom = qty.Mass(351534)
mtow = qty.Force.Kg(mtom)
wingspan = qty.Distance(64.8)
surface = qty.Area(436.8)
oswald = 0.77
cd_0 = 0.068
v_stall_mtow = qty.Velocity.Kts(154)

v_stall_to_lof = 1.15
v_stall_to_v2 = 1.18

stall_speeds: list[tuple[qty.Mass, qty.Velocity]] = [
    (mtom, v_stall_mtow),
    (qty.Mass.M_ton(350), qty.Velocity.Kts(153.5)),
    (qty.Mass.M_ton(345), qty.Velocity.Kts(151.5)),
    (qty.Mass.M_ton(340), qty.Velocity.Kts(149)),
    (qty.Mass.M_ton(335), qty.Velocity.Kts(147)),
    (qty.Mass.M_ton(330), qty.Velocity.Kts(145)),
    (qty.Mass.M_ton(325), qty.Velocity.Kts(142.5)),
    (qty.Mass.M_ton(320), qty.Velocity.Kts(140)),
    (qty.Mass.M_ton(315), qty.Velocity.Kts(137.5)),
    (qty.Mass.M_ton(310), qty.Velocity.Kts(135)),
    (qty.Mass.M_ton(305), qty.Velocity.Kts(132.5)),
    (qty.Mass.M_ton(300), qty.Velocity.Kts(130)),
    (qty.Mass.M_ton(295), qty.Velocity.Kts(127)),
    (qty.Mass.M_ton(290), qty.Velocity.Kts(124.5)),
    (qty.Mass.M_ton(285), qty.Velocity.Kts(121.5)),
    (qty.Mass.M_ton(280), qty.Velocity.Kts(118.5)),
    (qty.Mass.M_ton(275), qty.Velocity.Kts(115.5)),
    (qty.Mass.M_ton(270), qty.Velocity.Kts(112.5)),
    (qty.Mass.M_ton(265), qty.Velocity.Kts(109)),
    (qty.Mass.M_ton(260), qty.Velocity.Kts(103)),
]


## Constants
Before any distance is calculated, some constant values for this scenario will be calculated.

#### 1. Density with the given temperature and constant ISA pressure
$ \large \rho = \frac{P}{R \cdot T} $

#### 2. Thrust for one engine and with both
The engines are flat-rated up to $ ISA+15 \degree C $, since the temperature in this scenario is $ 17 \degree C $, the thrust does not decrease. The thrust values in LBS are converted to Newton using the qty module.

#### 3. Aspect ratio and $ C_{L_{max}} $

$ \large AR = \frac{b^2}{S} $

The $ C_{L_{max}} $ can be obtained using the lift equation with the MTOW and stall speed at MTOW:

$ \large C_{L_{max}} = \frac{2 \cdot W}{\rho \cdot v_{stall}^2 \cdot S} $


In [188]:
# 1. Density
density = pressure / (287 * temp)

# 2. Thrust
thrust_per_engine = qty.Force.Kg(qty.Mass.Lbs(104500))
thrust = qty.Force(thrust_per_engine * 2)

# 3. Aspect ratio and CL_max
aspect_ratio = wingspan**2 / surface
cl_max = (2 * mtow) / (density * surface * v_stall_mtow**2)

cl_max

2.0675363973645564

## Takeoff Ground Run
To calculate the take-off run distance, assuming the acceleration is not constant, three steps are taken:

#### 1. Calculate the optimal values of $ C_L $ and $ C_D $ for the minimum run distance

$ C_L = \frac{1}{2} \cdot \mu \cdot \pi \cdot AR \cdot e $

$ C_D = C_{D_0} + \frac{C_L^2}{\pi \cdot AR \cdot e} $

#### 2. Calculate the aerodynamic factor and d factor needed in the ground run equation

$ e = \frac{\rho \cdot S}{2 \cdot W} \cdot (C_D - \mu \cdot C_L) $

$ d = \frac{T}{W} - \mu $

#### 3. Calculate the ground run distance provided the acceleration is not constant

$ \large S_A = \frac{1}{2 \cdot g \cdot e} \cdot \ln{\frac{d}{d - e \cdot v_{LOF}^2}} $

In [189]:
def get_gnd_run(weight: qty.Force, v_stall: qty.Velocity) -> qty.Distance:
    # 1. Get the optimal C_L and C_D values
    cl_run = 0.5 * mu * math.pi * aspect_ratio * oswald
    cd_run = cd_0 + cl_run**2 / (math.pi * aspect_ratio * oswald)
    
    # 2. Calculate aero-factor and d-factor
    aero_factor = (density * surface) / (2 * weight) * (cd_run - mu * cl_run)
    d_factor = thrust / weight - mu

    # 3. Get the ground run distance
    v_lof = v_stall_to_lof * v_stall
    
    ground_run = 1 / (2 * 9.81 * aero_factor) * math.log(d_factor / (d_factor - aero_factor * v_lof**2))

    # Return the ground run as a distance for the given weight and stall speed
    return qty.Distance(ground_run)

## Takeoff Air Run

At lift-off speed, an N - 1 situation arises. Which means the available thrust will be that of one engine. This value will be used in the following calculations.

The takeoff arc distance is calculated in multiple steps:

#### 1. Calculate the $ C_L $ and C_D $ at point C
At point C, the end of the arc, the aircraft assumes a climb with the optimal configuration for climb gradient. In this configuration $\frac{C_L}{C_D} $ is maximum. 

The $ C_D $ in this configuration is obtained by:

$ C_{D_C} = 2 \cdot C_{D_0} $

and $ C_L $ is obtained by:

$ C_{L_C} = (C_{D_C} - C_{D_0}) \cdot \pi \cdot AR \cdot e $

#### 2. Calculate the drag at point C

The drag is obtained by

$ D_C = W \cdot \frac{C_{D_C}}{C_{L_C}} $

#### 3. Calculate the climb gradient at point C

While in the given configuration, the climb gradient is obtained by:

$ \sin{\gamma_C} = \frac{T - D}{W} $

#### 4. Calculate the lift and drag coefficients at liftoff

By equalling the lift at stall speed and the lift at take-off speed, it is found that:

$ \large C_{L_{LOF}} = \frac{C_{L_{max}}}{ratio^2} $

where $ ratio $ is equal to the ratio between the liftoff speed and the stall speed, 1.15 in this scenario. Then the drag coefficient is obtained using

$ \large C_{D_{LOF}} = C_{D_0} + \frac{C{L_{LOF}}^2}{\pi \cdot AR \cdot e} $

#### 5. Calculate the average drag during the takeoff arc
The drag at liftoff is obtained using the drag equation

$ D_{LOF} = \frac{1}{2} \cdot \rho \cdot v_{LOF}^2 \cdot S \cdot C_{D_{LOF}} $

Then the average drag is obtained by

$ D_{avg} = \frac{D_{LOF} + D_C}{2} $

#### 6. Calculate the air run distance

All known values can now be substituted into the energy equation to solve for the air run distance:

$ \large \frac{W}{2 \cdot g} \cdot (v_C^2 - v_{LOF}^2) = (T - D_{avg}) \cdot S_B - W \cdot h_B $

This becomes

$ \LARGE S_B = \frac{\frac{W}{2 \cdot g} \cdot (v_C^2 - v_{LOF}^2) + W \cdot h_B}{T - D_{avg}} $


In [190]:
def get_air_run(weight: qty.Force, v_stall: qty.Velocity) -> tuple[qty.Distance, float]:
    # 1. Get C_L and C_D at point C
    cd_c = cd_0 * 2
    cl_c = (cd_c - cd_0) * math.pi * aspect_ratio * oswald
    
    # 2. Calculate the drag at point C
    drag_c = weight * (cd_c / cl_c)
    
    # 3. Calculate the climb gradient at point C
    gradient_c = math.asin((thrust_per_engine - drag_c) / weight)
    
    # 4. Calculate the lift coefficent at liftoff
    cl_lof = cl_max / v_stall_to_lof**2
    cd_lof = cd_0 + cl_lof**2 / (math.pi * aspect_ratio * oswald)

    # 5. Calculate the average drag in the takeoff arc
    v_lof = v_stall * v_stall_to_lof
    drag_lof = 0.5 * density * v_lof**2 * surface * cd_lof
    drag_avg = (drag_lof + drag_c) / 2

    # 6. Calculate the air run distance
    v_c = v_stall * v_stall_to_v2
    air_run = (weight / (2 * 9.81) * (v_c**2 - v_lof**2) + weight * height_b) / (thrust_per_engine - drag_avg)

    # Return the air run distance for the given weight and stall speed
    return qty.Distance(air_run), gradient_c

## Obstacle Clearance

Obstacle 1600 m at 103 ft. After takeoff arc, aircraft climbs with optimal climb gradient $ \gamma_C $. This gives:

$ \tan{\gamma_C} = \frac{hor}{ver} \rightarrow ver = hor \cdot \tan{\gamma_C}$

In [191]:
def get_obstacle_clearance(residual_rwy_dist: qty.Distance, gradient_c: float) -> qty.Distance:
    height_at_obstacle = (obstacle_distance + residual_rwy_dist) * math.tan(gradient_c)
    return qty.Distance(height_at_obstacle - obstacle_height)

## Stop distance at $ V_1 $


In [192]:
def get_stop_dist(weight, v_stall):
    cl_run = cl_max / v_stall_to_lof**2
    cd_run = cd_0 + cl_run**2 / (math.pi * aspect_ratio * oswald)

    aero_factor = (density * surface) / (2 * weight)  * (cd_run - mu_brake * cl_run)
    d_factor = - mu_brake
    
    v_lof = v_stall * v_stall_to_lof
    stop_dist = 1 / (2 * 9.81 * aero_factor) * math.log((d_factor - aero_factor * v_lof**2) / d_factor)

    return qty.Distance(stop_dist)

In [193]:
ground_run = get_gnd_run(mtow, v_stall_mtow)
air_run, gradient_c = get_air_run(mtow, v_stall_mtow)

takeoff_dist = qty.Distance(ground_run + air_run)

residual_rwy_dist = qty.Distance(toda - takeoff_dist)

obstacle_clearance = get_obstacle_clearance(residual_rwy_dist, gradient_c)

stop_dist = get_stop_dist(mtow, v_stall_mtow)
asd = qty.Distance(ground_run + stop_dist)

print(takeoff_dist, stop_dist, obstacle_clearance, asd)

2814.21 meters 1526.36 meters 51.03 meters 3435.02 meters


In [194]:
for mass, v_stall in stall_speeds:
    weight = qty.Force.Kg(mass)
    ground_run = get_gnd_run(weight, v_stall)
    air_run, gradient_c = get_air_run(weight, v_stall)
    
    takeoff_dist = qty.Distance(ground_run + air_run)
    
    residual_rwy_dist = qty.Distance(toda - takeoff_dist)
    
    obstacle_clearance = get_obstacle_clearance(residual_rwy_dist, gradient_c)
    
    stop_dist = get_stop_dist(weight, v_stall)
    accel_stop_dist = qty.Distance(ground_run + stop_dist)
    
    print(mass, takeoff_dist, accel_stop_dist, obstacle_clearance)

351534.00 kilograms 2814.21 meters 3435.02 meters 51.03 meters
350000.00 kilograms 2770.20 meters 3397.45 meters 54.21 meters
345000.00 kilograms 2614.95 meters 3250.46 meters 65.60 meters
340000.00 kilograms 2448.45 meters 3075.27 meters 78.33 meters
335000.00 kilograms 2315.13 meters 2941.29 meters 90.03 meters
330000.00 kilograms 2190.14 meters 2812.49 meters 101.95 meters
325000.00 kilograms 2054.71 meters 2659.52 meters 115.20 meters
320000.00 kilograms 1928.32 meters 2514.25 meters 128.67 meters
315000.00 kilograms 1810.06 meters 2376.16 meters 142.40 meters
310000.00 kilograms 1699.13 meters 2244.80 meters 156.40 meters
305000.00 kilograms 1594.89 meters 2119.76 meters 170.73 meters
300000.00 kilograms 1496.77 meters 2000.68 meters 185.39 meters
295000.00 kilograms 1392.23 meters 1866.86 meters 201.33 meters
290000.00 kilograms 1305.69 meters 1759.82 meters 216.73 meters
285000.00 kilograms 1213.30 meters 1639.69 meters 233.42 meters
280000.00 kilograms 1126.68 meters 1526.37 me